# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [82]:
import sqlite3    # Python interface for SQLite databases
from IPython.utils.traitlets import Bool, Int, Unicode

In [83]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [84]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
NB_MASTER2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
NP_MASTER1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
NV_MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_NB_DATES_MASTER2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: QTE(NUM)
  4: LIEU(TEXT)
  5: REGION(TEXT)
NV_NP_MASTER1:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sqlite_sequence:
  0: name()
  1: seq()


From this point we will use __%%sql__ magic

In [85]:
%reload_ext sql
%sql sqlite:///wine.db

'Connected: None@wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [86]:
%%sql SELECT *
FROM Master1
LIMIT 10;

Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [87]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

First Sub-Table of Master1:

In [88]:
%sql SELECT NV, NP, QTE FROM Master1 WHERE NV!='None' AND NP!='None' LIMIT 5;

Done.


NV,NP,QTE
1,1,300
1,73,1
2,5,100
3,1,400
4,10,35


Second Sub-Table of Master1:

In [89]:
%sql SELECT NV, CRU, DEGRE, MILL FROM Master1 WHERE NV!='None' LIMIT 5;

Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980


Third Sub-Table of Master1:

In [90]:
%sql SELECT NP, NOM, PRENOM, REGION FROM Master1 WHERE NP!='None' LIMIT 5;

Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [91]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

Done.
Done.


[]

In [92]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
NB_MASTER2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
NP_MASTER1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
NV_MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
NV_NB_DATES_MASTER2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: QTE(NUM)
  4: LIEU(TEXT)
  5: REGION(TEXT)
NV_NP_MASTER1:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
dummy:
  0: DEGRE(NUM)
sqlite_sequence:
  0: name()
  1: seq()


In [93]:
%%sql
SELECT *
FROM dummy
LIMIT 5;

Done.


DEGRE
None
11.5
11.3
12.1
10.9


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [94]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [95]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [96]:
%%sql
DROP TABLE IF EXISTS NV_NP_MASTER1;

--- Create NV_NP_MASTER1 table
CREATE TABLE NV_NP_MASTER1 AS
SELECT NV, NP, QTE
FROM MASTER1
WHERE NV IS NOT NULL AND NP IS NOT NULL;

SELECT *
FROM NV_NP_MASTER1
LIMIT 5;

Done.
Done.
Done.


NV,NP,QTE
1,1,300
1,73,1
2,5,100
3,1,400
4,10,35


In [97]:
%%sql
DROP TABLE IF EXISTS NV_MASTER1;

--- Create NV_MASTER1 table
CREATE TABLE NV_MASTER1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM NV_MASTER1
LIMIT 5;

Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976


In [98]:
%%sql
DROP TABLE IF EXISTS NP_MASTER1;

--- Create NP_MASTER1 table
CREATE TABLE NP_MASTER1 AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

SELECT *
FROM NP_MASTER1
LIMIT 5;

Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne


#### Exercise 1.4

Create the new tables from Master2:

In [99]:
%%sql
DROP TABLE IF EXISTS NV_NB_DATES_MASTER2;

--- Create NV_NB_DATES_MASTER2 table
CREATE TABLE NV_NB_DATES_MASTER2 AS
SELECT NV, NB, DATES, QTE, LIEU, REGION
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL AND DATES IS NOT NULL;

SELECT *
FROM NV_NB_DATES_MASTER2
LIMIT 5;

Done.
Done.
Done.


NV,NB,DATES,QTE,LIEU,REGION
1,2,1977-11-02,33,BORDEAUX,NOUVELLE-AQUITAINE
1,44,2015-10-16,1,PARIS,ÎLE-DE-FRANCE
1,45,1983-12-31,1,RENNES,BRETAGNE
2,48,1983-12-25,2,LYON,AUVERGNE-RHÔNE-ALPES
3,7,1978-11-01,6,NICE,PROVENCE-ALPES-CÔTE D'AZUR


In [100]:
%%sql
DROP TABLE IF EXISTS NV_MASTER2;

--- Create NV_MASTER2 table
CREATE TABLE NV_MASTER2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

SELECT *
FROM NV_MASTER2
LIMIT 5;

Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976


In [101]:
%%sql
DROP TABLE IF EXISTS NB_MASTER2;

--- Create NB_MASTER2 table
CREATE TABLE NB_MASTER2 AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

SELECT *
FROM NB_MASTER2
LIMIT 5;

Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [102]:
%%sql
SELECT DISTINCT TYPE
FROM NB_MASTER2;

Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [103]:
%%sql
SELECT DISTINCT REGION
FROM NP_MASTER1
INNER JOIN NV_NP_MASTER1 ON NP_MASTER1.NP=NV_NP_MASTER1.NP
INNER JOIN NV_MASTER1 ON NV_MASTER1.NV=NV_NP_MASTER1.NV
WHERE CRU IN ('Pommard', 'Brouilly');

Done.


REGION
Bourgogne
Rhone


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [104]:
%%sql
SELECT DISTINCT REGION
FROM NP_MASTER1
INNER JOIN NV_NP_MASTER1 ON NP_MASTER1.NP=NV_NP_MASTER1.NP
INNER JOIN NV_MASTER1 ON NV_MASTER1.NV=NV_NP_MASTER1.NV
WHERE CRU='Pommard' AND REGION IN (SELECT DISTINCT REGION
                                   FROM NP_MASTER1
                                   INNER JOIN NV_NP_MASTER1 ON NP_MASTER1.NP=NV_NP_MASTER1.NP
                                   INNER JOIN NV_MASTER1 ON NV_MASTER1.NV=NV_NP_MASTER1.NV
                                   WHERE CRU='Brouilly');

Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [105]:
%%sql
SELECT CRU, MILL,QTE
FROM NV_NP_MASTER1
INNER JOIN NV_MASTER1 ON NV_MASTER1.NV=NV_NP_MASTER1.NV
GROUP BY CRU, MILL

Done.


CRU,MILL,QTE
Arbois,1976,69
Arbois,1980,675
Auxey Duresses,1914,350
Beaujolais Primeur,1983,80
Beaujolais Villages,1975,800
Beaujolais Villages,1976,500
Beaujolais Villages,1978,450
Beaujolais Villages,1979,100
Bellet,1976,45
Blanquette de Limoux,1978,110


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [106]:
%%sql
SELECT NV_NP_MASTER1.NV, NV_MASTER1.CRU, COUNT(NP) AS NB_PRODUCERS
FROM NV_NP_MASTER1
INNER JOIN NV_MASTER1 ON NV_MASTER1.NV=NV_NP_MASTER1.NV
GROUP BY NV_NP_MASTER1.NV HAVING COUNT(NP)>3;

Done.


NV,CRU,NB_PRODUCERS
45,Chiroubles,5
78,Etoile,5
89,Cotes de Provence,4
98,Corbieres,5


#### Exercise 2.6

Which producers have not produced any wine?

In [107]:
%%sql
SELECT NP_MASTER1.NP, NOM, PRENOM
FROM NP_MASTER1
LEFT JOIN NV_NP_MASTER1 ON NP_MASTER1.NP=NV_NP_MASTER1.NP
GROUP BY NP_MASTER1.NP HAVING SUM(QTE) IS NULL
LIMIT 5;

Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [108]:
%%sql
SELECT NB_MASTER2.NB, NOM, PRENOM
FROM NB_MASTER2
INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
INNER JOIN NV_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
WHERE MILL=1980
GROUP BY NB_MASTER2.NB

Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [109]:
%%sql
SELECT NB, NOM, PRENOM
FROM NB_MASTER2
WHERE NB NOT IN (SELECT NB_MASTER2.NB
                FROM NB_MASTER2
                INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
                INNER JOIN NV_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
                WHERE MILL=1980
                GROUP BY NB_MASTER2.NB)
ORDER BY NOM
LIMIT 5;

Done.


NB,NOM,PRENOM
9,Ajar,Emile
10,Andersen,Yann
7,Anouilh,Jean
4,Apollinaire,Guillaume
1,Aristote,None


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [110]:
%%sql
SELECT NB, NOM, PRENOM
FROM NB_MASTER2
WHERE NB NOT IN (SELECT NB
                 FROM NB_MASTER2
                 WHERE NB NOT IN (SELECT NB_MASTER2.NB
                                  FROM NB_MASTER2
                                  INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
                                  INNER JOIN NV_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
                                  WHERE MILL=1980
                                  GROUP BY NB_MASTER2.NB))
AND NB IN (SELECT NB_MASTER2.NB
            FROM NB_MASTER2
            INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
            INNER JOIN NV_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
            WHERE MILL=1980
            GROUP BY NB_MASTER2.NB)

Done.


NB,NOM,PRENOM
2,Artaud,Antonin
44,Gide,Andre
45,Giono,Jean
8,Aragon,Louis
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.10

List all wines from 1980

In [111]:
%%sql
SELECT NV, CRU
FROM NV_MASTER2
WHERE MILL=1980
GROUP BY CRU
ORDER BY CRU

Done.


NV,CRU
74,Arbois
26,Chateau Corton Grancey
82,Cornas
20,Cote de Brouilly
87,Cotes de Provence
78,Etoile
43,Fleurie
4,Mercurey
16,Meursault
84,Rasteau


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [112]:
%%sql
SELECT NOM, PRENOM, NV_MASTER2.NV, CRU, QTE
FROM NV_MASTER2
INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
INNER JOIN NB_MASTER2 ON NB_MASTER2.NB=NV_NB_DATES_MASTER2.NB
WHERE MILL=1980 AND NB_MASTER2.NB=2

Done.


NOM,PRENOM,NV,CRU,QTE
Artaud,Antonin,1,Mercurey,33


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [113]:
%%sql
SELECT NB_MASTER2.NB, NOM, PRENOM, COUNT(NV_MASTER2.NV) AS NB_WINES_2018
FROM NB_MASTER2
INNER JOIN NV_NB_DATES_MASTER2 ON NV_NB_DATES_MASTER2.NB=NB_MASTER2.NB
INNER JOIN NV_MASTER2 ON NV_NB_DATES_MASTER2.NV=NV_MASTER2.NV
WHERE MILL=1980 
GROUP BY NB_MASTER2.NB HAVING NB_WINES_2018=(SELECT COUNT(y.NV)
                                            FROM NV_MASTER2 y
                                            WHERE y.MILL=1980)


Done.


NB,NOM,PRENOM,NB_WINES_2018
44,Gide,Andre,18
